In [2]:
import gym
import random
import numpy as np
from tqdm import tqdm


In [3]:
env = gym.make("Taxi-v3",render_mode="ansi")
env.reset()

(369, {'prob': 1.0, 'action_mask': array([1, 1, 1, 0, 0, 0], dtype=int8)})

In [4]:
print(env.render())   
# environmenti görmek için

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




0: güney

1: kuzay

2: doğu

3: batı

4: yolcuyu almak

5: yolcuyu birak


In [5]:
action_space = env.action_space.n
print(action_space)
state_space = env.observation_space.n
print(state_space)

6
500


In [6]:
q_table = np.zeros((state_space,action_space))

In [9]:
alpha = 0.1 # learning rate
gamma = 0.6 # discount rate
# epsilon
epsilon = 0.1  # exploration rate

for i in tqdm(range(1,100001)):   # tqdm ilermleme çubuğunu vermek için
    
    state,_ = env.reset() 
    # her eğitim döngüüsnde ortamı sıfırlıyoruz
    
    done = False
   
    while not done:

        if random.uniform(0,1) < epsilon:  # exploration           
            action = env.action_space.sample()  # rastgele eylem
        
        else:  # exploit (öğrenilen en iyi eylemi seeçer)
            action = np.argmax(q_table[state])

        next_state, reward, done, info, _ = env.step(action)
        #seçilen eylemi uygular, ortamın ynei durumu, bu eylemin getirdği ödül, ortmamın bitip bitmediği ve bilgi döner, ek bilgi(genelde kullanılmaz) 

        q_table[state, action] = q_table[state, action] + alpha * (reward + gamma * np.max(q_table[next_state]) - q_table[state, action])

        state = next_state
        # yeni durumu güncelle, sonraki adıma geç

print("Training completed.\n")
print("Final Q-Table Values")
print(q_table)

        

100%|██████████| 100000/100000 [00:30<00:00, 3253.24it/s]

Training completed.

Final Q-Table Values
[[  0.           0.           0.           0.           0.
    0.        ]
 [ -2.41837064  -2.3639511   -2.41837065  -2.36395109  -2.27325184
  -11.36394979]
 [ -1.87014396  -1.45024     -1.87014399  -1.45024002  -0.7504
  -10.4502323 ]
 ...
 [ -1.10535148   0.41599699  -1.08547861  -1.22059642  -3.55544025
   -3.42556788]
 [ -2.17085139  -2.12207969  -2.13259586  -2.12207951  -3.64718909
   -5.72610044]
 [  2.47508347   0.85070756   3.74494653  11.          -2.80636754
   -2.7672456 ]]


TEST

In [10]:
total_epochs, total_penalties = 0, 0   # toplam adım ve ceza sayısı
episodes = 100
for _ in range(episodes):
    state, _ = env.reset()
    epochs, penalties, reward = 0, 0, 0
    done = False

    while not done:
        
        action = np.argmax(q_table[state])
        # her adımda Q Tablosuna göre en iyi eylemi seç (öğrenilen strateji)

        next_state, reward, done, info, _ = env.step(action)
        state = next_state
        # seçilen yelemi uygula, ortamı güncelle 

        if reward == -10 :
            penalties += 1
        epochs += 1
    total_epochs += epochs
    total_penalties += penalties

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs/episodes}")
print(f"Average penalties per episode: {total_penalties/episodes}")

Results after 100 episodes:
Average timesteps per episode: 13.04
Average penalties per episode: 0.0
